In [1]:
import os
import pandas as pd
from utils import all_image_paths, get_name, all_file_paths, get_names
import re
import json


In [2]:
"""
촬영된 이미지에서 1cm^2 당 픽셀의 크기 아래와 같다.  
1cm^ = 34 pixel x 34 pixel 
"""
unit_area = 34 * 34


In [3]:
# 폴더 경로
folder_names = ['JINDO ADULT BIG',
                'JINDO ADULT SMALL',
                'JINDO BABY BIG',
                'JINDO BABY SMALL',
                'WANDO ADULT BIG',
                'WANDO ADULT SMALL',
                'WANDO BABY BIG',
                'WANDO BABY SMALL']

In [4]:
map_fname_side = {'36.jpg': "f",
                  '37.jpg': "b",
                  '38.jpg': "f",
                  '39.jpg': "b",
                  '40.jpg': "f",
                  '41.jpg': "b",
                  '42.jpg': "f",
                  '43.jpg': "b",
                  '44.jpg': "f",
                  '45.jpg': "b",
                  '46.jpg': "f",
                  '47.jpg': "b",
                  '71.jpg': "f",
                  '72.jpg': "b",
                  '73.jpg': "f",
                  '74.jpg': "b",
                  '48.jpg': "f",
                  '49.jpg': "b",
                  '58.jpg': "f",
                  '59.jpg': "b",
                  '61.jpg': "f",
                  '62.jpg': "b",
                  '63.jpg': "f",
                  '65.jpg': "b",
                  '66.jpg': "f",
                  '67.jpg': "b",
                  '50.jpg': "f",
                  '51.jpg': "b",
                  '52.jpg': "f",
                  '53.jpg': "b",
                  '54.jpg': "f",
                  '55.jpg': "b",
                  '56.jpg': "f",
                  '57.jpg': "b",
                  '71.jpg': "f",
                  '72.jpg': "b",
                  '73.jpg': "f",
                  '74.jpg': "b",
                  '68.jpg': "f",
                  '69.jpg': "b"}

In [5]:
# 폴더 별 image path, anno_paths, json , anno(DataFrame) 을 생성합니다.
map_dirname_imapaths = {}
map_imgname_region = {}
map_imgname_imgpath = {}
map_imgname_size = {}
map_imgname_entity = {}
map_dirname_annopath = {}
map_dirname_json = {}  # 
map_dirname_anno = {}  # 

valid_anno_paths = []
raw_dataset_dir = '../datasets/raw'
for folder_name in folder_names:

    # folder 이름 규칙 : {Region} {ADULT or BABY} {SMALL or BIG } 
    region, entity, size = folder_name.split(' ')

    # 폴더 별 모든 이미지 경로를 추출, {폴더이름: [파일 경로, 파일 경로1, ...]}
    folder_path = os.path.join(raw_dataset_dir, folder_name)
    img_paths = all_image_paths(folder_path)
    map_dirname_imapaths[folder_name] = img_paths
    img_names = get_names(img_paths)

    # 
    map_imgname_imgpath = {name: path for name, path in zip(img_names, img_paths)}

    # image 별 region, entity, size 을 저장합니다.    
    for img_path in img_paths:
        img_name = get_name(img_path)
        map_imgname_region[img_name] = region
        map_imgname_size[img_name] = size
        map_imgname_entity[img_name] = entity

    # coco_imglab.json 으로 끝나는 파일경로를 찾아 반환 및 저장
    anno_dirpath = os.path.join(folder_path, 'anno')
    anno_paths = all_file_paths(anno_dirpath)
    pattern = re.compile(r'.*coco_imglab\.json$')
    valid_anno_path = [path for path in anno_paths if pattern.match(path)]
    # 리스트 형태여서 리스트를 제거한다. ['../../adfasf'] -> '../../adfasf'
    map_dirname_annopath[folder_name] = valid_anno_path[0]

    # coco format 을 json 으로 읽어와 python dict 변수에 저장
    f = open(valid_anno_path[0])
    json_obj = json.load(f)
    map_dirname_json[folder_name] = json_obj

    # json 을 DataFrame 로 변환 
    image_df = pd.DataFrame(json_obj['images'])
    map_imgid_fname = {id: fname for fname, id in zip(image_df['file_name'], image_df['id'])}
    sample_df = pd.DataFrame(json_obj['annotations'])

    # df 내 area 가 10 이하 오류 데이터는 삭제함
    mask = sample_df.area < 1
    sample_df = sample_df.loc[~mask, :]

    # Column 추가 df 내 image id 을 파일 명으로 교체함
    sample_df.loc[:, ['image_id']] = sample_df.loc[:, ['image_id']].replace(map_imgid_fname)
    sample_df['region'] = sample_df.loc[:, ['image_id']].replace(map_imgname_region)
    sample_df['size_'] = sample_df.loc[:, ['image_id']].replace(map_imgname_size)
    sample_df['entity'] = sample_df.loc[:, ['image_id']].replace(map_imgname_entity)
    sample_df['side'] = sample_df.loc[:, ['image_id']].replace(map_fname_side)
    sample_df['image_path'] = sample_df.loc[:, ['image_id']].replace(map_imgname_imgpath)
    sample_df['abs_area'] = sample_df.loc[:, ['area']] / unit_area
    
    # 이미지가 존재하지 않는 annotation 은 제거 합니다. (35.jpg, 60.jpg, 64.jpg 제거) -> (35.jpg, 61.jpg, 65.jpg 제거) 
    mask = sample_df.image_id.isin(img_names)
    sample_df = sample_df.loc[mask]

    # anno 저장
    map_dirname_anno[folder_name] = sample_df
assert len(map_dirname_annopath) == len(folder_names)


In [6]:
# 모든 anno_df 을 하나로 모음
anno_df = pd.concat(map_dirname_anno.values())
anno_df = anno_df.reset_index(drop=True)

In [7]:
anno_df.to_csv('../datasets/anno.csv')